In [2]:
%load_ext autoreload
%autoreload 2

from dataset import preprocess_data
import matplotlib.pyplot as plt
import numpy as np
import segmentation_models_pytorch as smp

import torch

data_path = "/home/ubuntu/same_size/"
num_classes = 3
batch_size = 1

In [3]:
train_data_loader, val_data_loader = preprocess_data(data_path, num_classes=num_classes, batch_size=batch_size)


validating dataset...
1227 images, 1227 labels
finished validating dataset.
Loading dataset from /home/ubuntu/same_size/ of length 1227
Train dataset has 982 batches of size 1
Validation dataset has 245 batches of size 1


In [4]:
for i, (img, mask) in enumerate(train_data_loader):

    print(i, img.shape, mask.shape)
    print(np.unique(mask))

    # fig, ax = plt.subplots(1, 3, figsize=(5, 5))
    # ax[0].imshow(img[0, 0], cmap="gray")
    # ax[1].imshow(mask[0, 0] == 1, cmap="gray")
    # ax[2].imshow(mask[0, 0] == 2, cmap="gray")
    # plt.show()

    if i > 25:
        break

0 torch.Size([1, 1, 1536, 1024]) torch.Size([1, 1, 1536, 1024])
[0. 1.]
1 torch.Size([1, 1, 1536, 1024]) torch.Size([1, 1, 1536, 1024])
[0. 1.]
2 torch.Size([1, 1, 1536, 1024]) torch.Size([1, 1, 1536, 1024])
[0. 1.]
3 torch.Size([1, 1, 1536, 1024]) torch.Size([1, 1, 1536, 1024])
[0. 1.]
4 torch.Size([1, 1, 1536, 1024]) torch.Size([1, 1, 1536, 1024])
[0. 1.]
5 torch.Size([1, 1, 1536, 1024]) torch.Size([1, 1, 1536, 1024])
[0. 1. 2.]
6 torch.Size([1, 1, 1536, 1024]) torch.Size([1, 1, 1536, 1024])
[0. 1.]
7 torch.Size([1, 1, 1536, 1024]) torch.Size([1, 1, 1536, 1024])
[0. 1. 2.]
8 torch.Size([1, 1, 1536, 1024]) torch.Size([1, 1, 1536, 1024])
[0. 1. 2.]
9 torch.Size([1, 1, 1536, 1024]) torch.Size([1, 1, 1536, 1024])
[0. 1.]
10 torch.Size([1, 1, 1536, 1024]) torch.Size([1, 1, 1536, 1024])
[0. 1.]
11 torch.Size([1, 1, 1536, 1024]) torch.Size([1, 1, 1536, 1024])
[0. 1.]
12 torch.Size([1, 1, 1536, 1024]) torch.Size([1, 1, 1536, 1024])
[0. 1.]
13 torch.Size([1, 1, 1536, 1024]) torch.Size([1, 1, 

In [6]:
model = smp.Unet(
    encoder_name="resnet18",
    encoder_weights="imagenet",
    in_channels=1,  # grayscale images
    classes=num_classes,  # background, needle, lamella
)

optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
# criterion = torch.nn.CrossEntropyLoss()
criterion = smp.losses.SoftCrossEntropyLoss()#torch.nn.CrossEntropyLoss()

from tqdm import tqdm

epochs = 2 
device = "cuda" if torch.cuda.is_available() else "cpu"

WANDB= False
from train import train

model.to(device)

# # training loop
# for epoch in tqdm(range(epochs)):
#         print(f"------- Epoch {epoch+1} of {epochs}  --------")
        
#         train(model, device, train_data_loader, criterion, optimizer, WANDB)
#         # validate(model, device, val_data_loader, criterion, WANDB)


Unet(
  (encoder): ResNetEncoder(
    (conv1): Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (1): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track

In [10]:
criterion = smp.losses.DiceLoss(mode="multiclass")
#smp.losses.SoftCrossEntropyLoss()#torch.nn.CrossEntropyLoss()


for i, (images, masks) in enumerate(train_data_loader):
    # set model to training mode
    model.train()


    images = images.to(device)
    masks = masks.type(torch.LongTensor)
    masks = masks.reshape(
        masks.shape[0], masks.shape[2], masks.shape[3]
    )  # remove channel dim
    masks = masks.to(device)

    outputs = model(images)#.to(device)
    # pred = torch.argmax(outputs, dim=1)
    
    # print(images.shape, masks.shape, outputs.shape) #, pred.shape)
    # print("output: ", torch.unique(outputs))
    # print("pred: ", pred.shape, np.unique(pred.detach().cpu()))

    # print(masks.device)
    # print(pred.device) 
    # 
    print(outputs.dtype, masks.dtype)   

    loss = criterion(outputs, masks)

    # backwards pass
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    # break
    if i > 5:
        break

torch.float32 torch.int64
torch.float32 torch.int64
torch.float32 torch.int64
torch.float32 torch.int64
torch.float32 torch.int64
torch.float32 torch.int64
torch.float32 torch.int64
